In [30]:
#Imports
import time
import requests
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import os
import urllib.request
from urllib.request import urlopen
import pyautogui
import lxml.html
# from pywebcopy import save_webpage

In [25]:

internal_urls = set()
external_urls = set()

In [26]:
def is_url_valid(url):
   #Checks if the url is valid or not
    parsedURL = urlparse(url)
    return bool(parsedURL.netloc) and bool(parsedURL.scheme)

#The above function works as follows. We check if the scheme is present and there is a value in the network location part
# url = "https://umd.edu/virusinfo"
# urlparse(url)
# ParseResult(scheme='https', netloc='umd.edu', path='/virusinfo', 
# params='', query='', fragment='')

In [27]:

#The function gives all urls
def get_all_urls(url):
    urls = set()
    # domain name of the URL without the protocol (umd.edu in this case)
    domain_name = urlparse(url).netloc
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    for a_tag in soup.findAll("a"):
        href = a_tag.attrs.get("href")
        if href == "" or href is None:
            #href is empty and we don't need that a element
            continue
        #if the link is not absolute, make it by joining relative to the base
        href = urljoin(url, href)
        parsed_href = urlparse(href)
        #constructing an absolute URL from parsed data
        href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path
        if not is_url_valid(href):
            #in valid url
            continue
        if href in internal_urls:
            #it is already in the set, so we don't need to add
            continue
        if domain_name not in href:
            #it is an external link. i.e
            # Check if it is already there 
            if href not in external_urls:
#                 print(f"[EXT] External link: {href}")
                external_urls.add(href)
            continue
#         print(".",end=" ")
        print(f"[INT] Internal link: {href}")

        urls.add(href)
        internal_urls.add(href)
    return urls


In [28]:
total_urls_visited = 0
def crawl(url, max_urls=50):
    #Max URL is just to decrease the time if there are a lot of pages.
    #The following code was openly available of github and I found this
    #idea useful to inhibit crawling time
    global total_urls_visited
    total_urls_visited += 1
    links = get_all_urls(url)
    for link in links:
        if total_urls_visited > max_urls:
            break
        crawl(link, max_urls=max_urls)


In [33]:
base_urls = ["https://eoss.asu.edu/health/announcements/coronavirus"]
parent_folder = "Doctoral"

In [23]:
for base_url in base_urls:
    parsedurl = urlparse(base_url)
    # base_url_text = base_url.split("//",1)[1]
    base_url_text = parsedurl.netloc+parsedurl.path
    print(base_url_text)
    # base_url_text_domain = base_url_text.split("/",1)[0]
    base_url_text_domain = parsedurl.netloc
    print(base_url_text_domain)
    crawl(base_url)
    print("[+] Total External links:", len(external_urls))
    print("[+] Total Internal links:", len(internal_urls))
    print("[+] Total:", len(external_urls) + len(internal_urls))
    
    
    count = 0
    last_count = 989874
    immediate_urls = set() #Linked Associated with the current page, https://umd.edu/virusinfo/ in this case

    for i in range(0,5):
        for url in internal_urls:
            if base_url_text in url:
                immediate_urls.add(url)
        count = len(immediate_urls)
        if(count == last_count):
            break
        last_count = count
        print(count,i)
        for immediate_url in immediate_urls:
            crawl(immediate_url)
            print("[+] NEW Total External links:", len(external_urls))
            print("[+] NEW Total Internal links:", len(internal_urls))
            print("[+] NEW Total:", len(external_urls) + len(internal_urls))
        
        
    def removeAfterN(yourStr, nth, occurenceOf):
        return occurenceOf.join(yourStr.split(occurenceOf)[:nth])
    
    final_immediate_urls = set()

    for immediate_url in immediate_urls:
        stripped_url = removeAfterN(immediate_url,2,"http")
        final_immediate_urls.add(stripped_url)
    #     final_immediate_urls.add(rest)
    #     print(rest)

    print(final_immediate_urls)
    total_count = len(final_immediate_urls)
    print(total_count)
    
    count = 0
#     parent_folder = "Doctoral" # Change on Top
    name_of_folder = f'{base_url_text_domain}_Screenshots'
    for i in final_immediate_urls:
        options = webdriver.ChromeOptions()
        options.headless = True
        driver = webdriver.Chrome(options=options,executable_path='chromedriver.exe') # #Local Path of chrome driver
        url = i
        file_name = url.replace(base_url,'')
        file_name = file_name.replace('/','')
        if file_name == '':
                file_name = "Home"
        count = count + 1
        print(f"{count} of {total_count}")
        print(f'Visiting {base_url_text}/{file_name}')
        print(f'...Taking a screenshot')
        driver.get(url)
        if not os.path.exists(f'{parent_folder}\\{name_of_folder}'):
            os.makedirs(f'{parent_folder}\\{name_of_folder}')
        S = lambda X: driver.execute_script('return document.body.parentNode.scroll'+X)
        driver.set_window_size(S('Width'),S('Height')) # May need manual adjustment                                                                                                                
        driver.find_element_by_tag_name('body').screenshot(f'{parent_folder}\\{name_of_folder}\\{file_name}.png')
        print(f'Screenshot of {file_name} page taken! \n')
    #     print("Saving HTML")
    #     pyautogui.hotkey('ctrl', 's')
    #     time.sleep(1)
    #     pyautogui.typewrite(f'{file_name}.html')
    #     pyautogui.hotkey('enter')
        driver.quit()
    
    print(f'Task Completed! Files stored in the {parent_folder}\\{name_of_folder} Folder')


    
    
    webpage_save = set()
    webpage_save_count = 0
    webpage_cannot_save = 0
    for i in final_immediate_urls:
        print(i)
        try:
            code=urlopen(i).getcode()
            webpage_save.add(i)
            webpage_save_count = webpage_save_count+1
    #         print(code)
        except urllib.error.HTTPError as e:
            if e.code not in (400,403):
                webpage_cannot_save = webpage_cannot_save + 1
    #             print("discarded")
                continue

    print(f'Webpages saved as HTML :{webpage_save_count}')
    print(f'Webpages unable to save as HTML :{webpage_cannot_save}')

    #     try:
    #         print(i)

    #         print("added")
    #     except urllib.error.HTTPError as e:
    #         if e.code in (400,403):
    #             print("discarded")
    #             continue
    
    webpage_save_counter_loop = 0
    for webpage in webpage_save:
        url = webpage
        file_name = url.replace(base_url,'')
        file_name = file_name.replace('/','')
        if file_name == '':
                file_name = "Home"
        urllib.request.urlretrieve (webpage, f"{parent_folder}\\{name_of_folder}\\{file_name}.html")
        webpage_save_counter_loop = webpage_save_counter_loop+1
        print(f'{webpage_save_counter_loop} of {webpage_save_count} - {webpage}')

    print("Saving Completed")

    
    pdf_count = 0
    pdf_urls = set()
    for webpage in webpage_save:
        if webpage[-4:] == ".pdf":
            url = webpage
            file_name = url.replace(base_url,'')
            file_name = file_name.replace('/','')
            if file_name == '':
                file_name = "Home"
            pdf_urls.add(webpage)
            print(webpage)
            r = requests.get(url, stream=True)
            with open(f'{parent_folder}\\{name_of_folder}\\{file_name}.pdf', 'wb') as fd:
                for chunk in r.iter_content(2000):
                    fd.write(chunk)
            pdf_count = pdf_count + 1

    print(f"Pdfs saved: {pdf_count}")
    print("------------------------END-------------------------------\n")

eoss.asu.edu/health/announcements/coronavirus
eoss.asu.edu
[+] Total External links: 260
[+] Total Internal links: 473
[+] Total: 733
5 0
[+] NEW Total External links: 260
[+] NEW Total Internal links: 473
[+] NEW Total: 733
[+] NEW Total External links: 260
[+] NEW Total Internal links: 473
[+] NEW Total: 733
[+] NEW Total External links: 260
[+] NEW Total Internal links: 473
[+] NEW Total: 733
[+] NEW Total External links: 260
[+] NEW Total Internal links: 473
[+] NEW Total: 733
[+] NEW Total External links: 260
[+] NEW Total Internal links: 473
[+] NEW Total: 733
{'https://eoss.asu.edu/health/announcements/coronavirus/faqs', 'https://eoss.asu.edu/health/announcements/coronavirus', 'https://eoss.asu.edu/health/announcements/coronavirus/april-30-2020', 'https://eoss.asu.edu/health/announcements/coronavirus/may-15-2020', 'https://eoss.asu.edu/health/announcements/coronavirus/may-13-2020'}
5
1 of 5
Visiting eoss.asu.edu/health/announcements/coronavirus/faqs
...Taking a screenshot
Screen

[INT] Internal link: https://www.apu.edu/covid-19/features/2019/
[INT] Internal link: https://www.apu.edu/articles/selecting-a-college-during-covid-19/
[INT] Internal link: https://www.apu.edu/articles/setting-up-a-functional-work-space-at-home-for-college-students/
[INT] Internal link: https://www.apu.edu/covid-19/posts/27418/
[INT] Internal link: https://www.apu.edu/covid-19/posts/27417/
[INT] Internal link: https://www.apu.edu/covid-19/posts/27416/
[INT] Internal link: https://www.apu.edu/articles/4-financial-tips-to-maximize-your-stimulus-check/
[INT] Internal link: https://www.apu.edu/covid-19/posts/27406/
[INT] Internal link: https://www.apu.edu/covid-19/posts/27405/
[INT] Internal link: https://www.apu.edu/covid-19/posts/27393/
[INT] Internal link: https://www.apu.edu/articles/how-to-stay-spiritually-active-from-home/
[INT] Internal link: https://www.apu.edu/articles/mph-students-grapple-with-public-health-crisis-unfolding-in-real-time/
[INT] Internal link: https://www.apu.edu/a

[+] NEW Total External links: 335
[+] NEW Total Internal links: 618
[+] NEW Total: 953
[+] NEW Total External links: 336
[+] NEW Total Internal links: 618
[+] NEW Total: 954
[+] NEW Total External links: 336
[+] NEW Total Internal links: 618
[+] NEW Total: 954
[INT] Internal link: https://www.apu.edu/campussafety/services/vehicleregistration/
[INT] Internal link: https://www.apu.edu/campussafety/
[+] NEW Total External links: 336
[+] NEW Total Internal links: 620
[+] NEW Total: 956
[+] NEW Total External links: 336
[+] NEW Total Internal links: 620
[+] NEW Total: 956
[+] NEW Total External links: 336
[+] NEW Total Internal links: 620
[+] NEW Total: 956
[+] NEW Total External links: 336
[+] NEW Total Internal links: 620
[+] NEW Total: 956
[+] NEW Total External links: 336
[+] NEW Total Internal links: 620
[+] NEW Total: 956
[+] NEW Total External links: 336
[+] NEW Total Internal links: 620
[+] NEW Total: 956
[+] NEW Total External links: 336
[+] NEW Total Internal links: 620
[+] NEW To

1 of 49
Visiting www.apu.edu/covid-19//posts27377
...Taking a screenshot
Screenshot of posts27377 page taken! 

2 of 49
Visiting www.apu.edu/covid-19//posts27376
...Taking a screenshot
Screenshot of posts27376 page taken! 

3 of 49
Visiting www.apu.edu/covid-19//posts27418
...Taking a screenshot
Screenshot of posts27418 page taken! 

4 of 49
Visiting www.apu.edu/covid-19//posts27329
...Taking a screenshot
Screenshot of posts27329 page taken! 

5 of 49
Visiting www.apu.edu/covid-19//posts27320
...Taking a screenshot
Screenshot of posts27320 page taken! 

6 of 49
Visiting www.apu.edu/covid-19//posts27417
...Taking a screenshot
Screenshot of posts27417 page taken! 

7 of 49
Visiting www.apu.edu/covid-19//posts27358
...Taking a screenshot
Screenshot of posts27358 page taken! 

8 of 49
Visiting www.apu.edu/covid-19//features2019
...Taking a screenshot
Screenshot of features2019 page taken! 

9 of 49
Visiting www.apu.edu/covid-19//resources
...Taking a screenshot
Screenshot of resources page

12 of 49 - https://www.apu.edu/covid-19/posts/27353/
13 of 49 - https://www.apu.edu/covid-19/posts/27449/
14 of 49 - https://www.apu.edu/covid-19/posts/27323/
15 of 49 - https://www.apu.edu/covid-19/posts/27375/
16 of 49 - https://www.apu.edu/covid-19/features/
17 of 49 - https://www.apu.edu/covid-19/posts/27406/
18 of 49 - https://www.apu.edu/covid-19/posts/27380/
19 of 49 - https://www.apu.edu/covid-19/posts/27355/
20 of 49 - https://www.apu.edu/covid-19/posts/27393/
21 of 49 - https://www.apu.edu/covid-19/posts/27357/
22 of 49 - https://www.apu.edu/covid-19/social-distancing/
23 of 49 - https://www.apu.edu/covid-19/posts/27339/
24 of 49 - https://www.apu.edu/covid-19/posts/27345/
25 of 49 - https://www.apu.edu/covid-19/posts/27416/
26 of 49 - https://www.apu.edu/covid-19/posts/27448/
27 of 49 - https://www.apu.edu/covid-19/posts/27327/
28 of 49 - https://www.apu.edu/covid-19/
29 of 49 - https://www.apu.edu/covid-19/posts/27421/
30 of 49 - https://www.apu.edu/covid-19/posts/27405/
31

[+] NEW Total External links: 428
[+] NEW Total Internal links: 697
[+] NEW Total: 1125
[+] NEW Total External links: 428
[+] NEW Total Internal links: 697
[+] NEW Total: 1125
[+] NEW Total External links: 428
[+] NEW Total Internal links: 697
[+] NEW Total: 1125
21 2
[+] NEW Total External links: 428
[+] NEW Total Internal links: 697
[+] NEW Total: 1125
[+] NEW Total External links: 428
[+] NEW Total Internal links: 697
[+] NEW Total: 1125
[INT] Internal link: http://www.barry.edu/includes/docs/prepare/covid-19-symptom-self-check-tool.pdf
[+] NEW Total External links: 428
[+] NEW Total Internal links: 698
[+] NEW Total: 1126
[+] NEW Total External links: 428
[+] NEW Total Internal links: 698
[+] NEW Total: 1126
[+] NEW Total External links: 428
[+] NEW Total Internal links: 698
[+] NEW Total: 1126
[+] NEW Total External links: 428
[+] NEW Total Internal links: 698
[+] NEW Total: 1126
[INT] Internal link: http://www.barry.edu/prepare
[+] NEW Total External links: 428
[+] NEW Total Inte

https://www.barry.edu/prepare/covid-19/resources.html
https://www.barry.edu/prepare/covid-19/university-access.html
https://www.barry.edu/prepare/covid-19/student-updates.html
http://www.barry.edu/prepare/covid-19/resources.html
https://www.barry.edu/prepare/covid-19/university-updates.html
http://www.barry.edu/prepare/covid-19/university-updates.html
http://www.barry.edu/prepare/covid-19/travel-restrictions.html
https://www.barry.edu/prepare/covid-19/
https://www.barry.edu/prepare/covid-19/resident-student-updates.html
https://www.barry.edu/prepare/covid-19/cares-act.html
http://www.barry.edu/prepare/covid-19/student-updates.html
https://www.barry.edu/prepare/covid-19/questions.html
http://www.barry.edu/prepare/covid-19/questions.html
http://www.barry.edu/prepare/covid-19/prospective-students-faq.html
Webpages saved as HTML :21
Webpages unable to save as HTML :0
1 of 21 - http://www.barry.edu/prepare/covid-19/
2 of 21 - https://www.barry.edu/prepare/covid-19/facstaff-updates.html
3 of

[+] NEW Total External links: 501
[+] NEW Total Internal links: 788
[+] NEW Total: 1289
[INT] Internal link: https://www.bellarmine.edu/studentsuccess
[INT] Internal link: https://www.bellarmine.edu/docs/default-source/academic-affairs/fail-and-withdrawal-policy-updates-to-support-students.pdf
[INT] Internal link: https://www.bellarmine.edu/library/staff
[INT] Internal link: https://www.bellarmine.edu/about/president/Communiques/a-message-to-our-senior-students
[INT] Internal link: https://www.bellarmine.edu/studentaffairs/careercenter/online-services
[INT] Internal link: https://www.bellarmine.edu/coronavirus/epic-toolbox
[INT] Internal link: https://www.bellarmine.edu/studentsuccess/advising/appointment
[INT] Internal link: https://www.bellarmine.edu/studentaffairs/counselingcenter/services
[INT] Internal link: https://www.bellarmine.edu/studentaffairs/disabilityservices/student-faq-online-accommodations
[INT] Internal link: https://www.bellarmine.edu/international-services/services/